In [1]:
import cv2
import numpy as np
import os
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.metrics import precision_score
from skimage import data, exposure
import sys
import Funciones
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier


In [2]:
# 1. Preprocesamiento
from Funciones import filtroMediana
def preprocesar_imagen(ruta_imagen):
    imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
    if imagen is None:  # Manejar imágenes no cargadas
        print(f"Error al cargar la imagen: {ruta_imagen}")
        return None
    imagen = cv2.resize(imagen, (120, 120))  # Redimensionar a 128x128
    imagen = Funciones.ampliarRangoDin(imagen)
    imagen = cv2.equalizeHist(imagen)        # Ecualización del histograma
    #imagen = Funciones.filtroMediana(imagen)
    #imagen = Funciones.gradiente(imagen)
    #imagen = imagen/np.max(imagen)
    #imagen = cv2.GaussianBlur(imagen, (5, 5), 0)
    #imagen = Funciones.umbralizacion(imagen)
    
    return imagen

In [3]:
# 2. Creación del dataset prueba
def crear_dataset(directorio_imagenes):
    """
    Crea un dataset de características y etiquetas a partir de un directorio.
    """
    X, y = [], []  # X para características, y para etiquetas
    for etiqueta, nombre_clase in enumerate(['cat', 'dog']):
        directorio_clase = os.path.join(directorio_imagenes, nombre_clase)
        for nombre_archivo in os.listdir(directorio_clase):
            ruta_imagen = os.path.join(directorio_clase, nombre_archivo)
            imagen = preprocesar_imagen(ruta_imagen)
            if imagen is None:  # Omitir imágenes no válidas
                continue
            #fd, imagen  = Funciones.extraccion_hog(imagen)
            #imagen = np.float32(imagen)
            imagen = cv2.cornerHarris(imagen,2,3,0.04)
            posiciones, valores = Funciones.maximos_locales(imagen)
            #posiciones = posiciones[valores > 0.01 * np.max(imagen)]
            #valores = valores[valores > 0.01 * np.max(imagen)]
            caracteristicas = Funciones.reducir_mat(16, posiciones, valores)
            X.append(caracteristicas.flatten())
            y.append(etiqueta)
    return np.array(X), np.array(y)

In [7]:
# 3. Clasificación
def clasificacion(y_real, y_predicha):
    nombres_clases = {0: "gatos", 1: "perros"}
    reporte = {}
    for clase in [0, 1]:
        precision = precision_score(y_real, y_predicha, labels=[clase], average="micro")
        reporte[nombres_clases[clase]] = {
            'precision': precision
        }
    # Imprimir clasificación
    print("Clasificación :")
    for nombre_clase, metricas in reporte.items():
        print(f"\nClase {nombre_clase}:")
        for metrica, valor in metricas.items():
            print(f"  {metrica}: {valor * 100:.2f} %")

In [6]:
#pipeline
train_dir = 'cat_dog_500/train'
test_dir = 'cat_dog_500/test'

print("Extrayendo características Harris...")
X_train, y_train = crear_dataset(train_dir)
X_test, y_test = crear_dataset(test_dir)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


print("Entrenando modelo Random forest...")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

print("Evaluando modelo...")
y_pred = rf_model.predict(X_test)
clasificacion(y_test, y_pred)

Extrayendo características Harris...
Entrenando modelo Random forest...
Evaluando modelo...
Clasificación :

Clase gatos:
  precision: 100.00 %

Clase perros:
  precision: 86.21 %
